## Imports and initialization

In [8]:
import os
import json

from dataverk import Client
from dataverk_vault import api as vault_api
from google.cloud import storage
from google.oauth2 import service_account

### Dataverk

In [9]:
dv = Client()
vault_api.set_secrets_as_envs()

### Google cloud

In [10]:
creds = service_account.Credentials.from_service_account_info(json.loads(os.environ["GOOGLE_CREDS"]))

In [11]:
client = storage.Client(creds.project_id, credentials=creds)

In [12]:
bucket = client.get_bucket("styrk-bucket")

## Read from database

In [15]:
df = dv.read_sql("postgres", "SELECT * FROM mytable")

INFO:jovyan:PostgresConnector:2021-01-07 15:47:57: Reading from PostgreSQL database: postgres
INFO:jovyan:PostgresConnector:2021-01-07 15:47:57: 266 records returned in 0.15033435821533203 seconds.


In [17]:
df.shape

(266, 3)

## Convert dataframe to pickle and upload to gcs

In [8]:
df.to_pickle("databasedump_pg.pkl")

In [9]:
blob = bucket.blob("airflow/databasedump_pg.pkl")

In [10]:
blob.upload_from_filename("databasedump_pg.pkl")